In [84]:
from datetime import date, timedelta, datetime
import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

df = pd.read_csv (r'./client_value.csv')
print (df)

   past_value  current_value
0         100            200


In [3]:
def update_client_value(df, new_value): 
    past_value=df.iloc[0][1]
    df.iloc[0][0] = past_value
    df.iloc[0][1] = new_value   
    return df

In [4]:
df = update_client_value(df,200)

In [5]:
df[['past_value','current_value']].to_csv('client_value.csv',index=False)

# Update Version 2 With Time

In [6]:
df_time = pd.read_csv (r'./client_valuev2.csv')
print (df_time)

   past_value  current_value                        time
0           0            500                  2022-05-24
1         500           5000  2022-05-24 17:34:06.964684


In [7]:
df2 = {'past_value': 0, 'current_value': 500, 'time': date.today()}
df_time = df_time.append(df2, ignore_index = True)
df_time

,past_value,current_value,time
0,0,500,2022-05-24
1,500,5000,2022-05-24 17:34:06.964684
2,0,500,2022-05-24


In [25]:
def update_client_valuev2(df, new_value, date): 
    past_value = df.iloc[-1][1]
    df2 = {'past_value': past_value, 'current_value': new_value, 'time': date}   
    df = df.append(df2, ignore_index = True)
    return df

In [26]:
df_time = update_client_valuev2(df_time,9000, date.today())

In [27]:
df_time

,past_value,current_value,time
0,0,500,2022-05-24
1,500,5000,2022-05-24 17:34:06.964684
2,0,500,2022-05-24
3,500,9000,2022-05-24 20:17:53.210195
4,9000,9000,2022-05-24


In [28]:
df_time[['past_value','current_value','time']].to_csv('client_valuev2.csv',index=False)

# Portfolio

In [29]:
df_portfolio = pd.read_csv (r'./portfolio.csv')

In [30]:
df_portfolio

,coin,percentage,spent
0,BTC-USD,0.206853,210
1,ETH-USD,0.100000,15


In [31]:
def update_portfolio(df, value, signal, coin):

    df_coin = yf.download(coin,
                      start=date.today(), 
                      progress=False,
                      interval='1m'
    )
    if signal == 'buy':
            if coin not in df['coin'].tolist():
                                p= float(value/df_coin[df_coin.index == df_coin.index.min()]['Open'])
                                df2 = {'coin': coin, 'percentage':p, 'spent':value}
                                df = df.append(df2, ignore_index = True)

            
            else:

                for i, coin_name in enumerate(df.coin):
                    if coin_name==coin:
                        df['percentage'][i] = df['percentage'][i] + (value/df_coin[df_coin.index == df_coin.index.min()]['Open'])
                        df['spent'][i] = df['spent'][i] + value

            
    if signal == 'sell': 
        for i, coin_name in enumerate(df.coin):
                    if coin_name==coin:
                        if float(df['percentage'][i] - (value/df_coin[df_coin.index == df_coin.index.min()]['Open'])) < 0: return 'error'
                        df['percentage'][i] = df['percentage'][i] - (value/df_coin[df_coin.index == df_coin.index.min()]['Open'])
                        df['spent'][i] = df['spent'][i] - value 
        if coin not in df['coin'].tolist():
            print('error: coin not found')
    return df

In [32]:
df_portfolio = update_portfolio(df_portfolio, 100, signal='buy', coin='BTC-USD')

C:\Users\Admin\AppData\Local\Temp/ipykernel_2988/1955119821.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['percentage'][i] = df['percentage'][i] + (value/df_coin[df_coin.index == df_coin.index.min()]['Open'])
C:\Users\Admin\AppData\Local\Temp/ipykernel_2988/1955119821.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['spent'][i] = df['spent'][i] + value


In [135]:
df_portfolio

,coin,percentage,spent
0,ETH-USD,84.817972,167578


In [136]:
df_portfolio[['coin','percentage','spent']].to_csv('portfolio.csv',index=False)

In [141]:
date.today()-timedelta(days=30)

datetime.date(2022, 4, 24)

In [156]:
def total_value(df,date_bought):
    total = 0 
    for i, v in enumerate(df['coin']):
        df_coin = yf.download(v,
                      start=date_bought, 
                      progress=False,
                      interval='1d'
                    )

        coin_value = df_coin[df_coin.index == df_coin.index.min()]['Open']
        total = total + df['percentage'][i]*coin_value
    
    return float(total)


In [159]:
total_value(df_portfolio, date.today()-timedelta(days=10))

166253.6199668871

# Database generator

In [389]:
# para este caso as funções anteriores foram mudadas para 1d
# caso se pretenda voltar a usar para o dia presente será necessário repor o 1m no intervalo. 
# as datas devem tambem ser modificadas para date.today()
# AVISO NA FUNÇÃO TOTAL VALUE É NECESSÁRIO ACRESCENTAR [0] NA PARTE DO COIN VALUE (tambem falta algures no update portfolio)
# o spent está mal. Devia subtrair ao valor do portfolio dele não ao valor que ele gastou

In [390]:
def update_portfolio(df, value, signal, coin, date_bought):

    df_coin = yf.download(coin,
                      start=date_bought, 
                      progress=False,
                      interval='1d'
    )
    # conversion     
    new_date_bought = datetime.strptime(str(date_bought), '%Y-%m-%d')

    if signal == 'buy':
            if coin not in df['coin'].tolist():
                                p= float(value/df_coin[df_coin.index == new_date_bought]['Open'])
                                df2 = {'coin': coin, 'percentage':p, 'spent':value}
                                df = df.append(df2, ignore_index = True)

            
            else:
                for i, coin_name in enumerate(df.coin):
                    if coin_name==coin:
                        df['percentage'][i] = df['percentage'][i] + (value/df_coin[df_coin.index == new_date_bought]['Open'])
                        df['spent'][i] = df['spent'][i] + value

            
    if signal == 'sell': 
        for i, coin_name in enumerate(df.coin):
                    if coin_name==coin:
                        if float(df['percentage'][i] - (value/df_coin[df_coin.index == new_date_bought]['Open'][0])) < 0: return 'error'
                        df['percentage'][i] = df['percentage'][i] - (value/df_coin[df_coin.index == new_date_bought]['Open'])
                        df['spent'][i] = df['spent'][i] - value # ISTO ESTÁ MAL 
        if coin not in df['coin'].tolist():
            return 'error'
    return df

def total_value(df,date_bought):
    total = 0 
    for i, v in enumerate(df['coin']):
        #print('nome da coin no total value'+v)
        df_coin = yf.download(v,
                      start=date_bought, 
                      progress=False,
                      interval='1d'
                    )

        new_date_bought = datetime.strptime(str(date_bought), '%Y-%m-%d')
        coin_value = df_coin[df_coin.index == new_date_bought]['Open'][0]
        total = df['percentage'][i]*coin_value
        #print("total"+str(total))
    
    return float(total)

def update_client_valuev2(df, new_value,investment, signal, date): 
    last_invest = df.iloc[-1][2]
    if signal=='sell':
        investment = -abs(investment)    
    investment = last_invest+investment
    past_value = df.iloc[-1][1]
    df2 = {'past_value': past_value, 'current_value': new_value, 'spent': investment, 'time': date}   
    df = df.append(df2, ignore_index = True)
    return df


In [391]:
import random

In [395]:
df_time = pd.read_csv (r'./client_valuev2.csv')
df_portfolio = pd.read_csv (r'./portfolio.csv')

In [396]:
"""df_portfolio = update_portfolio(df_portfolio, investment, signal, coin_name)
portfolio_value = total_value(df_portfolio)
df_time = update_client_valuev2(df_time, portfolio_value)
df_portfolio[['coin','percentage','spent']].to_csv('portfolio_test.csv',index=False)
df_time[['coin','percentage','time']].to_csv('client_valuev2_test.csv',index=False)"""

"df_portfolio = update_portfolio(df_portfolio, investment, signal, coin_name)\nportfolio_value = total_value(df_portfolio)\ndf_time = update_client_valuev2(df_time, portfolio_value)\ndf_portfolio[['coin','percentage','spent']].to_csv('portfolio_test.csv',index=False)\ndf_time[['coin','percentage','time']].to_csv('client_valuev2_test.csv',index=False)"

In [398]:
iterations = 100
days_to_buy = 300
interval = 30
margin_price = 500
i = 1

while i < 6: 
    investment = random.randint(1, margin_price)
    signal = random.choice(['buy', 'sell'])
    date_bought = date.today()-timedelta(days=days_to_buy)
    coin_name = random.choice(['BTC-USD', 'ETH-USD', 'LUNA-USD', 'SOL-USD'])
    #print(i,investment,signal, coin_name)
    print(date_bought, signal)

    if i == 1:
        signal = 'buy'
        df_portfolio = update_portfolio(df_portfolio, investment, signal, coin_name, date_bought)
        print(df_portfolio)
        portfolio_value = total_value(df_portfolio, date_bought)
        df2 = {'past_value': 0, 'current_value': portfolio_value,'spent':investment, 'time': date_bought}   
        df_time = df_time.append(df2, ignore_index = True)
        df_portfolio[['coin','percentage','spent']].to_csv('portfolio_test.csv',index=False)
        df_time[['past_value','current_value','time']].to_csv('client_valuev2_test.csv',index=False)  
        days_to_buy=days_to_buy-interval
        i+=1

    else:    
        if isinstance(update_portfolio(df_portfolio, investment, signal, coin_name, date_bought), str): 
            while isinstance(update_portfolio(df_portfolio, investment, signal, coin_name, date_bought), str):
                investment = random.randint(1, margin_price)
                signal = random.choice(['buy', 'sell'])
                date_bought = date.today()-timedelta(days=days_to_buy)
                coin_name = random.choice(['BTC-USD', 'ETH-USD'])

            
        df_portfolio = update_portfolio(df_portfolio, investment, signal, coin_name, date_bought)
        print(df_portfolio)
        portfolio_value = total_value(df_portfolio, date_bought)
        df_time = update_client_valuev2(df_time, new_value=portfolio_value, investment=investment, signal=signal,date= date_bought)
        df_portfolio[['coin','percentage','spent']].to_csv('portfolio_test.csv',index=False)
        df_time[['past_value','current_value','spent','time']].to_csv('client_valuev2_test.csv',index=False)
        days_to_buy=days_to_buy-interval
        i+=1
            
    
    

2021-07-29 sell
       coin   percentage spent
0   BTC-USD     0.043607  2176
1   SOL-USD     0.195122    39
2  LUNA-USD  3611.445491    39
2021-08-28 buy
       coin   percentage spent
0   BTC-USD     0.043607  2176
1   SOL-USD    11.195121  1007
2  LUNA-USD  3611.445491    39
2021-09-27 buy
       coin   percentage spent
0   BTC-USD     0.043607  2176
1   SOL-USD    11.195121  1007
2  LUNA-USD  3611.445491    39
3   ETH-USD     0.151998   466
2021-10-27 buy
       coin    percentage spent
0   BTC-USD      0.043607  2176
1   SOL-USD     11.195121  1007
2  LUNA-USD  90635.191215   827
3   ETH-USD      0.151998   466
2021-11-26 sell
       coin    percentage spent
0   BTC-USD      0.027630  1234
1   SOL-USD     11.195121  1007
2  LUNA-USD  90635.191215   827
3   ETH-USD      0.151998   466


# 2nd Try

In [399]:
df_time = pd.read_csv (r'./client_valuev2.csv')
df_portfolio = pd.read_csv (r'./portfolio2.csv')

In [400]:
df_portfolio

,coin,quantity
0,BTC-USD,10


# New Idea 

In [401]:
your_account = 100000
coin_name = 'BTC-USD'
investment= 10000
signal = 'buy'

In [404]:
def translate_quantity(coin_name, investment):
    df_coin = yf.download(coin_name,
                      start=date.today(), 
                      progress=False,
                      interval='1m')
    coin_value = (investment/df_coin[df_coin.index == df_coin.index.min()]['Open'][0])

    return coin_value
    

In [405]:
translate_quantity('BTC-USD', 30000)

1.0140681489446923

In [424]:
account_coins = {'BTC-USD', 3}

In [429]:
def update_tuple(account_coins, coin_name, investment): 
    for coins in account_coins: 
        print(coins)
        if coins.get_key() == coin_name: 
           new_value = translate_quantity('BTC-USD', investment)
           print(new_value)
           coins[1] = coins[1]+new_value



In [422]:
account_coins

[('BTC-USD', 3)]